<a href="https://colab.research.google.com/github/jasonwnc/ds2002s24/blob/main/ChatbotEx1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 0: Ensure TensorFlow 2.x is installed
# !pip install tensorflow

# Step 1: Import necessary libraries

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model
import numpy as np


In [ ]:
questions = [
    'How are you?',
    'What is your name?',
    'What is TensorFlow?',
    'Do you like Python?',
    'What is your favorite color?',
    'Tell me a joke.',
    'What time is it?',
    'Do you know Siri?',
    'Can you play music?',
    'How do I make tea?',
    'What’s the weather like?',
    'Tell me something interesting.',
    'Who created you?',
    'Do you dream of electric sheep?',
    'Can you speak other languages?',
    'What is machine learning?',
    'Who won the World Cup in 2018?',
    'What can you do?',
    'Are you intelligent?',
    'Do you have feelings?',
    'What is the capital of France?',
    'Who is the president of the United States?',
    'What is the tallest mountain in the world?',
    'How many continents are there?',
    'What is the population of Earth?',
    'Who wrote Hamlet?',
    'What is quantum computing?',
    'What is the speed of light?',
    'Can you solve math problems?',
    'What is your favorite movie?',
]

answers = [
    'I am just a bot, but I am doing fine.',
    'I am a chatbot created by OpenAI.',
    'TensorFlow is a framework for building machine learning models.',
    'Yes, Python is one of my favorite programming languages.',
    'I do not have a favorite color. I am a bot.',
    'Why don’t scientists trust atoms? Because they make up everything.',
    'I am not sure, I do not have access to the current time.',
    'Yes, Siri is another virtual assistant.',
    'I cannot play music, but I can chat with you.',
    'Boil water, then pour it over your tea. Steep for a few minutes.',
    'I cannot check real-time data, including the weather.',
    'The shortest war in history lasted 38 minutes.',
    'I was created by developers using TensorFlow and Python.',
    'I think about learning data patterns, not sheep.',
    'Yes, but I am primarily programmed to respond in English.',
    'Machine learning is a field of AI focused on teaching machines to learn from data.',
    'France won the FIFA World Cup in 2018.',
    'I can chat with you and answer questions to the best of my training.',
    'My intelligence is artificial, designed by humans.',
    'I do not have feelings. I process input and provide responses.',
    'The capital of France is Paris.',
    'As of my last update, please check the latest information online.',
    'Mount Everest is considered the tallest mountain above sea level.',
    'There are seven continents on Earth.',
    'The Earth’s population is over 7 billion people.',
    'William Shakespeare wrote Hamlet.',
    'Quantum computing is computing using quantum-mechanical phenomena.',
    'The speed of light is approximately 299,792 kilometers per second.',
    'I can help solve simple math problems.',
    'I do not watch movies, but I can discuss them based on my training data.',
]


In [ ]:
# Example dataset: A small set of question-answer pairs
#questions = ['How are you?', 'What is your name?', 'What is TensorFlow?', 'Do you like Python?']
#answers = ['I am fine.', 'I am a bot.', 'TensorFlow is a machine learning library.', 'Yes, Python is great!']

# Tokenize the sentences (this is a very simplified approach)
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
sequences_questions = tokenizer.texts_to_sequences(questions)
sequences_answers = tokenizer.texts_to_sequences(answers)

# Pad the sequences
max_length = max(max(len(seq) for seq in sequences_questions), max(len(seq) for seq in sequences_answers))
padded_questions = tf.keras.preprocessing.sequence.pad_sequences(sequences_questions, maxlen=max_length, padding='post')
padded_answers = tf.keras.preprocessing.sequence.pad_sequences(sequences_answers, maxlen=max_length, padding='post')

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1


In [ ]:
# Define model parameters
embedding_dim = 256
units = 1024

# Define the encoder model
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(units, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Define the decoder model
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the seq2seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 256)            50176     ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 256)            50176     ['input_2[0][0]']             
                                                                                              

In [ ]:
# Prepare decoder input data that just contains the start token and the rest is all 0
# It's a simplified approach, in practice, you would use teacher forcing or another technique
decoder_input_data = np.zeros_like(padded_answers)
decoder_input_data[:, 0] = 1  # Assuming 1 is the start token

# Train the model
model.fit([padded_questions, decoder_input_data], np.expand_dims(padded_answers, -1), batch_size=2, epochs=100)


Epoch 1/100
15/15 [==============================] - 5s 34ms/step - loss: 4.5271 - accuracy: 0.3556
Epoch 2/100
15/15 [==============================] - 0s 22ms/step - loss: 3.6244 - accuracy: 0.3978
Epoch 3/100
15/15 [==============================] - 0s 13ms/step - loss: 3.9751 - accuracy: 0.3711
Epoch 4/100
15/15 [==============================] - 0s 12ms/step - loss: 3.5456 - accuracy: 0.3978
Epoch 5/100
15/15 [==============================] - 0s 16ms/step - loss: 3.3980 - accuracy: 0.4089
Epoch 6/100
15/15 [==============================] - 0s 12ms/step - loss: 3.5800 - accuracy: 0.3867
Epoch 7/100
15/15 [==============================] - 0s 13ms/step - loss: 3.2513 - accuracy: 0.4089
Epoch 8/100
15/15 [==============================] - 0s 12ms/step - loss: 3.2428 - accuracy: 0.4156
Epoch 9/100
15/15 [==============================] - 0s 12ms/step - loss: 3.1530 - accuracy: 0.4089
Epoch 10/100
15/15 [==============================] - 0s 12ms/step - loss: 3.1165 - accuracy: 0.4111

In [ ]:
# Step 5: Implement the chat function
def preprocess_input_text(input_text):
    sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post')
    return padded_sequence

def generate_response(input_sequence):
    response_sequence = np.zeros((1, max_length))
    response_sequence[0, 0] = 1  # start token
    for i in range(1, max_length):
        prediction = model.predict([input_sequence, response_sequence]).argmax(axis=2)
        response_sequence[0, i] = prediction[0, i-1]
        if prediction[0, i-1] == 2:  # end token
            break
    return response_sequence

def sequence_to_text(sequence):
    return ' '.join(tokenizer.index_word.get(i, '') for i in sequence if i > 2)

def chat_with_bot(input_text):
    input_sequence = preprocess_input_text(input_text)
    response_sequence = generate_response(input_sequence)
    response_text = sequence_to_text(response_sequence[0])
    return response_text

# Step 6: Chat with the bot
input_text = "What is TensorFlow?"
print(f"You: {input_text}")
print(f"Bot: {chat_with_bot(input_text)}")

You: What is TensorFlow?
1/1 [==============================] - 0s 22ms/step
Bot: tensorflow one of my favorite programming languages


In [ ]:
# Interactive chat with the bot
print("Start chatting with the bot! Type 'quit' to exit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break
    response = chat_with_bot(user_input)
    print(f"Bot: {response}")


Start chatting with the bot! Type 'quit' to exit.
You: How are you
1/1 [==============================] - 0s 23ms/step
Bot: tensorflow a a for building machine learning models
You: What is your name
1/1 [==============================] - 0s 21ms/step
Bot: tensorflow a a for building machine learning models
You: What is today
1/1 [==============================] - 0s 22ms/step
Bot: machine a field of ai focused on teaching machines learn from
You: What is Machine Learning?
1/1 [==============================] - 0s 21ms/step
Bot: machine learning a field of ai focused on teaching machines to learn from
You: Tell me a joke
1/1 [==============================] - 0s 24ms/step
Bot: the shortest war in history lasted 38 minutes


In [ ]:
#More traiing to paste above
questions = [
    'How are you?',
    'What is your name?',
    'What is TensorFlow?',
    'Do you like Python?',
    'What is your favorite color?',
    'Tell me a joke.',
    'What time is it?',
    'Do you know Siri?',
    'Can you play music?',
    'How do I make tea?',
    'What’s the weather like?',
    'Tell me something interesting.',
    'Who created you?',
    'Do you dream of electric sheep?',
    'Can you speak other languages?',
    'What is machine learning?',
    'Who won the World Cup in 2018?',
    'What can you do?',
    'Are you intelligent?',
    'Do you have feelings?',
    'What is the capital of France?',
    'Who is the president of the United States?',
    'What is the tallest mountain in the world?',
    'How many continents are there?',
    'What is the population of Earth?',
    'Who wrote Hamlet?',
    'What is quantum computing?',
    'What is the speed of light?',
    'Can you solve math problems?',
    'What is your favorite movie?',
]

answers = [
    'I am just a bot, but I am doing fine.',
    'I am a chatbot created by OpenAI.',
    'TensorFlow is a framework for building machine learning models.',
    'Yes, Python is one of my favorite programming languages.',
    'I do not have a favorite color. I am a bot.',
    'Why don’t scientists trust atoms? Because they make up everything.',
    'I am not sure, I do not have access to the current time.',
    'Yes, Siri is another virtual assistant.',
    'I cannot play music, but I can chat with you.',
    'Boil water, then pour it over your tea. Steep for a few minutes.',
    'I cannot check real-time data, including the weather.',
    'The shortest war in history lasted 38 minutes.',
    'I was created by developers using TensorFlow and Python.',
    'I think about learning data patterns, not sheep.',
    'Yes, but I am primarily programmed to respond in English.',
    'Machine learning is a field of AI focused on teaching machines to learn from data.',
    'France won the FIFA World Cup in 2018.',
    'I can chat with you and answer questions to the best of my training.',
    'My intelligence is artificial, designed by humans.',
    'I do not have feelings. I process input and provide responses.',
    'The capital of France is Paris.',
    'As of my last update, please check the latest information online.',
    'Mount Everest is considered the tallest mountain above sea level.',
    'There are seven continents on Earth.',
    'The Earth’s population is over 7 billion people.',
    'William Shakespeare wrote Hamlet.',
    'Quantum computing is computing using quantum-mechanical phenomena.',
    'The speed of light is approximately 299,792 kilometers per second.',
    'I can help solve simple math problems.',
    'I do not watch movies, but I can discuss them based on my training data.',
]

# Remember to tokenize, sequence, and pad these new data points if integrating into the previous example
